## A classic problem

Here I'm going to consider the half-pipe problem seen in Classical Mechanics by Taylor, Example 1.2. It consists of a skateboard in a semicircular half-pipe that is released at some angle $\phi$ from equilibrium position. This problem is analogous to many others in classical mechanics, like the simple pendulum problem, a mass on a spring, etc. For the moment, we'll neglect friction. 

Through a straightforward analysis of the forces, we can arrive at the equation of motion for the angle the skateboard makes with the bottom of the pipe:

\begin{equation} \tag{1}
\ddot{\phi} = -\frac{g}{R}\sin (\phi)
\end{equation}

where $g$ is the gravitational constant, and $R$ is the radius of the pipe. 

This differential equation has no solution in terms of commonly known elementary functions (though complex analysis says otherwise - see 
[this](https://mathworld.wolfram.com/JacobiEllipticFunctions.html) and [this](https://en.wikipedia.org/wiki/Jacobi_elliptic_functions#Jacobi_elliptic_functions_as_solutions_of_nonlinear_ordinary_differential_equations)).
However, using the small angle approximation, we can approximate the sine as $\sin(\phi) \approx \phi$, so that the equation becomes

\begin{equation} \tag{2}
\ddot{\phi} = - \frac{g}{R}\phi
\end{equation}

This has the well-known solution consisting of sines and cosines: $\phi(t) = A\sin(\omega_0 t) + B\cos(\omega_0 t)$, with $A$ and $B$ being constants determined by the boundary conditions of the system, and $\omega_0^2 = \frac{g}{R}$ being the natural frequency of oscillations. 

In the following, I will be using numerical integration (Runge Kutta 4) to compare the solution provided by the small angle approximation from Equation 2 to the numerical solution to the nonlinear Equation 1. 

In [1]:
import numpy as np
import sympy as sp
import scipy as sc
import matplotlib.pyplot as plt
import ipywidgets as wd